<a href="https://colab.research.google.com/github/Fuenfgeld/Agent_Tutorial_PydanticAI/blob/main/03_SystemPromts_Tools_PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install pydantic-ai
%pip -q install nest_asyncio
%pip -q install logfire

In [ ]:
import os
from google.colab import userdata

keyAntropic = userdata.get('Claude')
keyOpenAI = userdata.get('openAI')
keyLogFire = userdata.get('logfire')


os.environ["OPENAI_API_KEY"] = keyOpenAI
os.environ["ANTHROPIC_API_KEY"] = keyAntropic

import nest_asyncio
nest_asyncio.apply()
#logfire.configure(token=keyLogFire)

# System Promt

Generally, system prompts fall into two categories

   * Static system prompts: These are known when writing the code and can be defined via the system_prompt parameter of the Agent constructor.
   * Dynamic system prompts: These depend in some way on context that isn't known until runtime, and should be defined via functions decorated with @agent.system_prompt.



In [ ]:
import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from datetime import date

# Define the model
model = OpenAIModel('gpt-4o-mini')

# Create an agent that will use GPT-4o as the LLM
# The deps_type=str means we'll pass a string as dependency (patient ID)
agent = Agent(
    model,
    deps_type=str,
    system_prompt="Use the patient's ID when discussing their medical information.",
)

# Add system prompt that includes the patient ID in the context
@agent.system_prompt
def add_patient_id(ctx: RunContext[str]) -> str:
    return f"The patient's ID you're discussing is {ctx.deps}."

# Add system prompt that includes today's date for medical record keeping
@agent.system_prompt
def add_current_date() -> str:
    return f'Today\'s date for the medical record is {date.today()}.'

# Run the agent with a query about lab results and include patient ID
result = agent.run_sync('What were my latest blood glucose results? also what date is it', deps='PT12345')
print(result.output)


# Function Tools
Pydantic AI function tools are a core feature of the PydanticAI framework, allowing AI agents to call external Python functions-known as "tools"-to retrieve information or perform actions during a conversation. This system enables agents to go beyond simple text generation and interact with structured data, APIs, or any Python-accessible resource

## Schema Extraction

When you define a tool with parameters and docstrings, PydanticAI will extract both the parameter types and descriptions to build a detailed schema for the LLM:

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
import random


def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    import random
    return str(random.randint(1, 6))

model = OpenAIModel('gpt-4o-mini')
agent = Agent(model, tools=[roll_die])

result = agent.run_sync('Roll the die for me')
print(result.output)
#> The result is 4.


Tools and plain tools in PydanticAI both enable AI agents to call Python functions during a conversation, but they differ in how they interact with the agent's context and dependencies.
  * Tools

    Registered using the @agent.tool decorator or by specifying takes_ctx=True when creating a Tool object.

    Accept a RunContext parameter as their first argument. This context provides access to dependencies or other runtime information that the agent may inject for each run.

    Useful when a function needs to use external data, session state, or any information managed by the agent at runtime.

    Example:


  * Plain Tools

    Registered using the @agent.tool_plain decorator or by specifying takes_ctx=False when creating a Tool object.

    Do not accept a RunContext parameter; they are standard Python functions with only their explicit arguments.

    Use these when the tool does not need any agent context or dependencies-just the parameters provided in the function call.

    Example:

In [ ]:

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
import random


# Define the model
model = OpenAIModel('gpt-4o-mini')



agent = Agent(
    model,
    deps_type=str,
    system_prompt=(
        "You're a dice game, you should roll the die and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Use the player's name in the response."
    ),
)


@agent.tool_plain
def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


@agent.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


dice_result = agent.run_sync('My guess is 4', deps='Max')


print(dice_result.output)
#> Congratulations Anne, you guessed correctly! You're a winner!

In [ ]:
from datetime import date
import os
from colorama import Fore
import logfire
from pydantic_ai import Agent, RunContext, Tool
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel
from dotenv import load_dotenv



# Configure logging for medical system
logfire.configure(token=keyLogFire)

# Define the AI model to use
model = OpenAIModel('gpt-4o-mini')

# Define the output model for clinical protocol evaluation
class ProtocolQuality(BaseModel):
    """Clinical Protocol Quality metrics"""
    evidence_level: float  # Evidence level score (1-5)
    guideline_adherence: float  # Percentage of adherence to clinical guidelines
    review: str  # Clinical review comments

# Tool to retrieve the clinical protocol
def get_clinical_protocol(ctx: RunContext[str]) -> str:
    """Get the clinical protocol text"""
    with logfire.span('Agents calls tool with context') as span:
        span.set_attribute('context', ctx.deps)
    return f"The clinical protocol is: {ctx.deps}"

# Tool to get the medical standards
def get_medical_standards() -> ProtocolQuality:
    """Get the medical standards for clinical protocols"""
    return ProtocolQuality(evidence_level=4.0, guideline_adherence=90.0, review="These are the current medical standards")

# Protocol creation agent
protocol_agent = Agent(model=model,
                       system_prompt="You are an experienced clinician. Create a clinical protocol according to the user's requirements. Return only the protocol text.")

# Protocol review agent
protocol_review_agent = Agent(model=model,
                              tools=[Tool(get_clinical_protocol, takes_ctx=True),
                                     Tool(get_medical_standards, takes_ctx=False)],
                              result_type=ProtocolQuality,
                              system_prompt="You are an experienced medical director. You are reviewing a clinical protocol to ensure it meets quality standards. Provide protocol quality metrics and a review comparing it to medical standards.")

# Run the agents
with logfire.span('Calling Protocol Agent') as span:
    result = protocol_agent.run_sync("Create a diabetes management protocol for primary care that includes HbA1c monitoring schedule.")
    span.set_attribute('result', result.output)

print(Fore.YELLOW, result.output)

with logfire.span('Calling Review Agent') as span:
    result = protocol_review_agent.run_sync("Review this protocol and provide quality metrics.", deps=result.output)
    span.set_attribute('result', result.output)
print(Fore.CYAN, result.output)